In [90]:
import seaborn as sns
import pandas as pd
mock=pd.read_csv("mock_db.csv")

In [91]:
def normalize(dataset):
    columns=dataset.columns
    for col in columns:
        maxValue=dataset[col].max()
        dataset[col]=(dataset[col]/maxValue)*100
    return dataset

In [92]:
def normalizeColumn(col):
    maxValue=col.max()
    return (col/maxValue)*100

In [93]:
def URMS(dataset):
    dataset=normalize(dataset)
    columns=dataset.columns
    numcolumns=len(columns)
    w=[0 for i in columns]
    corr=dataset.corr()
    for i in range(numcolumns):
        for j in range(i+1,numcolumns):
            corrBetweenIandJ=corr[columns[i]][columns[j]]
            if corrBetweenIandJ>0:
                if w[i]>=0 and w[j]>=0:
                    w[i]+=corrBetweenIandJ
                    w[j]+=corrBetweenIandJ
                elif w[i]<0 and w[j]<0:
                    w[i]-=corrBetweenIandJ
                    w[j]-=corrBetweenIandJ
                elif w[i]>=0 and w[j]<0:
                    w[i]-=corrBetweenIandJ
                    w[j]+=corrBetweenIandJ
                else :
                    w[i]+=corrBetweenIandJ
                    w[j]-=corrBetweenIandJ
            else:
                if w[i]>=0 and w[j]>=0:
                    if w[i]<w[j]:
                        w[i]+=corrBetweenIandJ
                        w[j]-=corrBetweenIandJ
                    else:
                        w[i]-=corrBetweenIandJ
                        w[j]+=corrBetweenIandJ
                        
                elif w[i]<0 and w[j]<0:
                    if w[i]<w[j]:
                        w[i]+=corrBetweenIandJ
                        w[j]-=corrBetweenIandJ
                    else:
                        w[i]-=corrBetweenIandJ
                        w[j]+=corrBetweenIandJ
                        
                elif w[i]>=0 and w[j]<0:
                    w[i]-=corrBetweenIandJ
                    w[j]+=corrBetweenIandJ
                else :
                    w[i]+=corrBetweenIandJ
                    w[j]-=corrBetweenIandJ
    return w

In [100]:
def rank(column,dataset):
    dataset=dataset.copy(deep=True)
    weights=URMS(dataset.drop(columns=column,axis=1))
    columns=dataset.drop(column,axis=1).columns
    dataset["Score"]=0
    for index,col in enumerate(columns):
        dataset["Score"]=dataset["Score"]+normalizeColumn(dataset[col])*weights[index]
    return dataset.sort_values(by="Score",ascending=False)


In [101]:
rank("Country",mock)

,Country,GDP,LEB,IMR,Tub,Score
7,USA,41674,77.93,2,7,158.971177
3,Ireland,38058,79.40,6,4,43.543033
6,UK,31580,79.30,6,5,36.139700
2,Germany,30496,79.48,3,4,31.580082
4,Italy,27750,81.18,3,4,15.399186
0,Finland,30469,79.09,3,3,10.432643
1,France,29644,80.47,6,4,2.924626
5,Spain,27270,80.28,13,4,-60.282417
